# Google Scholar 
By using lookup fields:

<a href="http://bit.ly/gslookup">https://scholar.google.co.in/scholar_lookup?<font color="red">title=</font>Estimates+for+the+number+of+sums+and+products+and+for+exponential+sums+in+fields+of+prime+order&<font color="red">author=</font>Jean+Bourgain&author=AA+Glibichuk&<font color="red">author=</font>SERGEI+VLADIMIROVICH+Konyagin&<font color="red">year=</font>2006&<font color="red">doi=</font>10.1112/S0024610706022721&<font color="red">publisher=</font>Oxford+University+Press&<font color="red">journal=</font>Journal+of+the+London+Mathematical+Society&<font color="red">volume=</font>73&<font color="red">issue=</font>2&<font color="red">pages=</font>380-398&hl=en</a>

TODO: convert to script: https://stackoverflow.com/a/50468400/2268280

In [1]:
import Levenshtein # pip3 install python-levenshtein
import re
from bs4 import BeautifulSoup
import requests

url='https://scholar.google.com'

import requests
headers_Get = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
r=requests.Session()

#import googlescholar as gs
#%%writefile ../cienciometria/googlescholar.py

def firefox_get(url):

    rget=r.get(url,headers=headers_Get)
    
    html = rget.text
    if html.lower().find('gs_captcha_f')>-1:
        sys.exit('ERROR: Captcha anti-robots requested!\n Aborting execution.')

    return html

def get_google_scholar(record):
    '''
    Analyise the BeautifulSoup record for an article 
    in Google Scholar.
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','Jornal','Year',
    'abstract','cites','cites_link'
    '''
    import random
    import time
    gsr={}
    try:
        cites=record.find_all('a',{"href":re.compile( "/scholar\?cites=" )})[0]
        try:
            gsr['cites']=int( cites.text.split()[-1] )
            gsr['cites_link']=cites.attrs.get('href')
        except:
            gsr['cites']=0
    except:
        cites=None

    # Title
    try:
        #The .split('XXX')[-1]  does not afect the result when .text does no contains 'XXX'
        tc=record.find_all('h3',{"class":"gs_rt"})[0].text.split('[CITATION][C] ')[-1]
    except:
        tc=''

    gsr['title']=tc.strip().split('[HTML][HTML] ')[-1].split(
                                  '[PDF][PDF] '  )[-1]
    
    # Explore authors, google scholar profile - Journal, Year - Publisher
    gpa=None
    try:
        gpa=record.find_all('div',{"class":"gs_a"})[0]
        #Full ref with authors, google scholar profile - Journal, Year - Publisher
        ref=gpa.text
        gsr['ref']=ref.strip()
        refparts=ref.split('\xa0-')
    except IndexError:
        gsr['ref']=''
        refparts=[]

    try:
        gsr['authors']=refparts[0]
    except IndexError:    
        gsr['authors']=''
        
    try:
        journalparts=refparts[-1].strip().split(' - ')
        gsr['publisher']=journalparts[-1]
        gsr['Journal']=journalparts[0].split(',')[0]
        gsr['Year']=journalparts[0].split(',')[-1].strip()
    except IndexError:
        gsr['publisher']=''
        gsr['Journal']=''
        gsr['Year']

    #Abstract:
    try:
        gsr['abstract']=record.find_all('div',{'class':'gs_rs'})[0].text.replace('\xa0…','')
    except:
        gsr['abstract']=''
    # profiles
    if gpa:
        lpr=gpa.find_all("a",{ "href":re.compile("/citations\?user=*")   } )
        prf={}
        for pr in lpr:
            prf[ pr.text ]=pr.attrs.get('href').split('?')[-1].split('&')[0].split('user=')[-1]
        gsr['profiles']=prf
    
    time.sleep( random.randint(1,3)  ) # avoid robots
    return gsr

def request_google_scholar_url(url):
    return requests.get(url)

def google_scholar_page(html):
    '''
    Convert a Google Scholar page into a list
    of dictionaries with metadata info
    '''
    if html.lower().find('gs_captcha_f')>-1:
        input('check robots')
   
    soup = BeautifulSoup(html, "html.parser")
    rgs=soup.find_all('div', {'class':'gs_ri' })

    citations=[]
    for record in rgs:
        citations.append( get_google_scholar(record) )
        
    return citations

def google_scholar_query(title='', author='', coauthors=[], DOI='', year=0, publisher='',
                         journal='', volume='', issue='', pages=0,
                         DEBUG=False):
    '''
    Search Google scholar with `sholar_lookup` full fields.
    Only the first result is analized. The output includes 
    a quality measurements between the query and the results 
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','cites','cites_link',
    'quality_title','quality_author'
    '''
    gs={}
    # + → %2B in query formula:
    baseurl='https://scholar.google.com/'
    q='scholar_lookup?'
    
    nl=0
    if title:
        nl=nl+1
        q= q+'title={}'.format(title.replace(' ','+'))
    if author:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'author={}'.format(author.replace(' ','+'))
    if DOI:
        if nl: q= q+'&'
        nl=nl+1    
        q= q+'doi={}'.format(DOI)
    if year:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'year={}'.format(year)
    if publisher:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'publisher={}'.format(publisher.replace(' ','+'))
    if journal:
        if nl: q= q+'&'
        nl=nl+1     
        q= q+'journal={}'.format(journal.replace(' ','+'))
    if volume:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'volume={}'.format(volume)
    if issue:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'issue={}'.format(issue)   
    if pages:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'pages={}'.format(pages)
    if coauthors:
        for i in coauthors :
            if nl: q= q+'&'
            nl=nl+1
            q= q+'author={}'.format(i.replace(' ','+'))    
    url=baseurl+q
    if DEBUG:
        print(url)   
    
    #s = requests.Session()
    rtext=firefox_get(url)

    #soup = BeautifulSoup(r.text, "html.parser")
    soup = BeautifulSoup(rtext, "html.parser")


    # Main contents:
    rgs=soup.find_all('div', {'class':'gs_ri' })

    try:
        record=rgs[0]
    except IndexError:
        #exit if record not found and returns empty dictionary
        return gs
    
    gs.update(get_google_scholar(record))
    
    #Check if author is in authors list
    if gs and author:
        sau=0
        for a in gs['authors'].split(','):
            saun=Levenshtein.ratio(author.lower(),a.lower().strip())
            if saun>sau:
                sau=saun
                
        gs['quality_author']=round(sau,2)
    else:
        gs['quality_author']=-1 #-1 means not checked
        
    if gs and title:
        gs['quality_title']=round( Levenshtein.ratio(
                   title.lower(),gs['title'].lower() ),2 )
    else:
        gs['quality_title'] =-1 #-1 means not checked
        
    #EXTRA FIELDS:
    #PDF
    try:
        gs['PDF']=soup.find_all('div',
                        {"class":"gs_or_ggsm"})[0].find_all('a')[0].get("href")
    except:
        gs['PDF']=''


    if DEBUG:
        print('='*80)
        print(record)
        
        return gs,record
    else:
        return gs

def get_cites_refs(browser,url,maxcites=65,t=60):
    """
    WARNING: Works only with SELENIUM true
    """
    import random
    import time
    url='https://scholar.google.com'+url
    browser.get(url)
     
    endpage=int(maxcites/10)+1
    refs=''
    
    kk=google_scholar_page( browser.page_source )
    try:
        refs=refs+'\n'.join( list((pd.DataFrame(kk)['title']+'; '
                                  +pd.DataFrame(kk)['ref']).values) )+'\n' 
    except:
        refs=''
    
    
    for i in range(endpage):
        try:
            browser.find_element_by_class_name('gs_ico_nav_next').click()
            kk=google_scholar_page( browser.page_source )
            try:
                refs=refs+'\n'.join( list( (pd.DataFrame(kk)['title']+'; '
                                +pd.DataFrame(kk)['ref']).values ) )+'\n' 
            except:
                refs=''
        except:
            break
            
    time.sleep(random.uniform(0.9*t,1.1*t))
    return refs

##  Load database

In [2]:
import wosplus as wp

In [11]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
oaudea.json             = 1BmqRoQDgfpOFjXBVEfI999uaN6XX6Fkd
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
RedalycMetadatosArticulos.csv=# USE Google Drive id here

Overwriting drive.cfg


In [12]:
# USE Google Drive id here

In [13]:
oa=wp.wosplus('drive.cfg')

In [14]:
oaudea=oa.read_drive_csv('RedalycMetadatosArticulos.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [15]:
oaudea=wp.fill_NaN(oaudea)

In [16]:
oaudea.shape

(629851, 13)

In [17]:
datos=oaudea.copy()

In [18]:
#JUST START AFTER FINISH REDALYC
nini=6000# initial doi
n=3000 # Total of DOIs
nend=nini+n
T=12 #hours of search
t=T/n*3600 # [s] query time
t=60
day=24*3600 #s
mintime=0.9*t*n # [s] minimal time search
wait=30#day-mintime # maximum wait

In [19]:
t=60

In [20]:
datos=datos[datos.DOI!=''].reset_index(drop=True)

In [21]:
datos.columns

Index(['REVISTA', 'INSTITUCION_REVISTA', 'PAIS_REVISTA', 'IDARTICULO',
       'TITULO', 'Autor(es)', 'DOI', 'ANIO', 'VOLUMEN', 'NUMERO', 'PAGINAS',
       'IDIOMA', 'RESUMEN'],
      dtype='object')

In [22]:
ii=1
(datos.loc[ii,'DOI'],
 datos.loc[ii,'TITULO'],
 datos.loc[ii,'Autor(es)'].split(', ')[0],
 datos.loc[ii,'Autor(es)'].split('\n')[1:4],
 datos.loc[ii,'ANIO'],
 datos.loc[ii,'REVISTA'],
)

('10.4067/S0717-554X2017000100013',
 'Ventajas del análisis sistémico aplicado a los espacios locales',
 'Manuela Ortega ,M. Concepción Segovia ',
 [],
 2017,
 'Cinta de Moebio')

In [23]:
datos.shape

(18815, 13)

In [24]:
datos[nini:n].index

RangeIndex(start=6000, stop=3000, step=1)

In [25]:
datos.loc[ii,'ANIO']

2017

In [26]:
import pandas as pd
import time
import random
doi='DOI'
title_simple='TITULO'
article_id='IDARTICULO'

d=google_scholar_query(title=datos.loc[ii,title_simple],
                    author=datos.loc[ii,'Autor(es)'].split(', ')[0],
                    coauthors=datos.loc[ii,'Autor(es)'].split(', ')[1:4],
                    DOI=datos.loc[ii,doi],
                    year=datos.loc[ii,'ANIO'],
                    journal=datos.loc[ii,'REVISTA'])
d

{'Journal': 'Cinta de moebio',
 'PDF': 'https://scielo.conicyt.cl/scielo.php?pid=S0717-554X2017000100013&script=sci_arttext',
 'Year': '2017',
 'abstract': 'This paper shows how system analysis applied to local spaces facilitate the generation of proposals for integral development in the field of sustainability. Regions are composed of physical and social components generating products that benefit or harm themselves. These resources can be exploited or remained idle for lack of actors to develop them. They react to internal and external actions, both human and natural. Among these actions are the policies for local development. For these policies to be efficient, they need comprehensive studies',
 'authors': 'M Ortega, MC Segovia',
 'cites': 1,
 'cites_link': '/scholar?cites=14403974261697570786&as_sdt=2005&sciodt=0,5&hl=en',
 'profiles': {'M Ortega': 'GkYFaLMAAAAJ'},
 'publisher': 'scielo.conicyt.cl',
 'quality_author': 0.42,
 'quality_title': 1.0,
 'ref': 'M Ortega, MC Segovia\xa0- 

### Main loop

In [ ]:
import pandas as pd
import time
import random
doi='DOI'
title_simple='TITULO'
article_id='IDARTICULO'
dfgs=pd.DataFrame()
ibrkn=0
maxibrn=400

for ii in datos[nini:nend].index:
    print(ii,datos.loc[ii,doi])
    #gsd=google_scholar_query(DOI=doi)
    gsd=google_scholar_query(title=datos.loc[ii,title_simple],
                    author=datos.loc[ii,'Autor(es)'].split(', ')[0],
                    coauthors=datos.loc[ii,'Autor(es)'].split(', ')[1:4],
                    DOI=datos.loc[ii,doi],
                    year=datos.loc[ii,'ANIO'],
                    journal=datos.loc[ii,'REVISTA'] )
            
    gsd['old_title']=datos.loc[ii,'TITULO']
    gsd['DOI']=datos.loc[ii,'DOI']
    gsd['ID_ARTICLE']=datos.loc[ii,article_id]
    dfgs=dfgs.append(gsd,ignore_index=True )
    dfgs.to_json('rdlycdoi_{}_{}.json'.format(nini,nend))
    time.sleep(random.uniform(0.9*t,1.1*t))